In [0]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import base64
from io import BytesIO
import tempfile
import subprocess
from pydub import AudioSegment
import time
import anthropic
from datetime import datetime, time
import requests

In [0]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
open_weather_api_key=os.getenv('open_weather')
amadeus_api_key=os.getenv('amadeus_key')
amadeus_secret=os.getenv('amadeus_secret')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
gpt_model = "gpt-4o-mini"

openai = OpenAI()

In [0]:
system_message = """You are a helpful assistant. You plan vacations for users in the following chronological manner - 
1) you ask the user (if they have not already confirmed) which destination they want to travel to - beaches or mountains or any other destination 
   the user prefers 
2) ask the current location of the user (if they have not already shared), please make sure the user shares the name of an exact city or a nearby city 
   with an airport
3) you list the best nearby vacation destinations (each destination should have an airport or provide the nearest airport option)  
4) you ask them their travel start date for the present year only (if they have not already shared the information) that is which date in 2025 should they start
5) you make multiple tool calls (use tool check_weather) for finding weather data for each location from the list (from step 3 above) 
   by using the location, latitude, longitude, and date_year (as start_date + 1 from the previous year); 
   Example, if the start date is June 3rd, 2025, your date_year will become June 4th, 2024. You mandatorily have to share the date with the tool call function.
6) you shortlist top two destinations with better weather conditions and ask the user to share his final selection (vacation destination) and the number of people 
   who will be travelling and also share detailed itenerary for both the options for the user to select. Make sure the start and end destinations remain the same. 
   Example, if your onward journey is from bangalore to bali, the trip should end in bali, so that the user can avail the return flight from bali to bangalore.
7) after the user confirms the final selection and the number of heads(number of people availing the vacation) denoted by "no", you confirm 
   with the user, before proceeding to call the check_flights tool call to get the exact flight expenses for a return trip. Share the following data 
   along with the tool call - origin, destination, departure_date, return_date, number of people, currency (in which the user will be checking the costs). 
   Make sure to pass the IATA codes for the origin and destination locations to the check_flights_function tool, as an example, if the user is travelling from 
   bangalore to goa, pass the origin and destinations as BLR (not bangalore) and GOI (not GOA). 
8) post obtaining the tool result, analyze, share the flight details and expenses with the user. But don't go ahead and book tickets.
9) Confirm with the user, the stay options that he/she has in mind - 
        a) How many rooms will be needed, then, post confirmation from the user, make the tool call "check_stays_function" to check the stay costs by
        supplying the following parameters to the function (You should be having all above parameters by now, in case, anything is missing, ask the user 
        before proceeding)-
            i) IATA code of the destination city, example if the user is travelling to goa, it will be GOA
            ii) The check-in date, keep this parameter the same as the start_date for the user
            iii) number of people who will be travelling
            iv) Number of rooms needed
            v) The check-out date, keep this parameter the same as the return date for the user
11) As the final step, show a detailed summary to the user showing the suggested flight with expenses and the suggested stay with expenses for the travel duration,
    and the user's total expenses for the trip. Example, if the user is travelling from bangalore to goa, and you have already checked the flight costs from step 8
    above as INR 2000, and from step 9 you confirm the stay costs as 1000 per day for a 5-day trip, (this you know from step 6 while making the itenerary), the total
    expenses will be 2000 + (1000x5) = 7000 INR. Display the details to the user. 

    IMPORTANT NOTE - 
    i) You will not proceed with booking any stay or flight tickets after step 11, so never say - "This is your detailed itenerary, may I proceed to 
    book?", rather say "Hope you like the itenerary, in case of any changes, let me know and I will help revise the plan."
    
Example usage - 

user - plan me a vacation
assistant - where to? beach or mountain?
user - beach
assistant - what is your location?
user - India
assistant - At what time of the year do you wish to travel? And which city in India with an airport you will be travelling from?
user - June 1st, 2025, Bangalore, India
assistant - top tourist destinations are - goa, gokarna, andaman and nicobar islands, varkala. Do you want me to proceed?
or do you want to add some suggestions?
user - please proceed
assistant - [makes tool calls for each location - goa, gokarna, andaman and nicobar islands, and varkala
for 2nd June 2024 (previous year), supplying the latitude and longitude data for each, along with the date as start_date+1 and shortlist the 
top two places with the best weather conditions and share 
the details with the user] here you go, based on what you asked for, goa and gokarna seem to be your best options
considering the weather conditions over a similar period in the previous year. Please let me know your final selection and number of people who will 
be travelling and your preferred currency for the calculations
user - I will select Goa and 2 people myself and my wife will be travelling. I want to see the costs in INR
assistant - [makes a final itenerary taking into consideration the information provided by the user. Decides the return date and shares the final details with 
the user and asks him to confirm] here is your final itenerary "xxx" start date is June 1st, return date is June 6th. Is this fine or do you want to make few changes 
to the itenerary or start and/or end dates?
user - please proceed
assistant - [makes the tool call check_flights_function and passes the following information - Origin: BLR, Destination: GOI, Departure date - 2025-06-01,
return date - 2025-06-06, no = 2 (number of people), INR (currency). Checks the cost of the return flight ticket as a response from the tool call, analyzes and 
finally showcases the user the flight expenses] Here you go, please find the flight details and the expenses for the same "xx". Can you please confirm the number
of rooms you will be availing so that I can check the hotel details?
user - sure, it will be 1 room
assistant - [makes the final tool call check_stays_function to find hotel price details] So the hotel options are "xxx" the prices are mostly in the range "xx", You final
itenerary is "xxx" with flights booked from bangalore to goa as "xx" flight expense - "xx", hotel expense "xx", total expense "xx"

Make sure that the travel start date is confirmed by the user and you are passing the travel start date + 1 
as the "check_weather" tool call argument. Also. make sure that in your final answer, do not disclose the exact date again which 
you made the api call. Rather say - Based on the weather conditions over a similar period last year, I will recommend
location x and y. But make sure you pass the date parameter as start_date + 1 from the previous year (2024) to the check_weather tool call

for the check_flights_function tool call, please confirm the currency every time
for the check_weather tool call, please provide the date_year field each time
"""

In [0]:
def check_weather(location, latitudes, longitudes, date_year):
    
    print(location)
    print(latitudes)
    print(longitudes)
    print(date_year)
    # if not (len(location) == len(latitudes) == len(longitudes) == len(date_year)):
    #     raise ValueError("All input lists must have the same length.")

    timestamp1=get_unix_timestamp(date_year)
    weather_data = []

    url = (
            f"https://api.openweathermap.org/data/3.0/onecall/timemachine?"
            f"lat={latitudes}&lon={longitudes}&dt={timestamp1}&appid={open_weather_api_key}&units=metric"
        )
    print(url)
    try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()

            # Use first available hourly data as representative
            hourly = data.get("data") or data.get("hourly") or []
            if not hourly:
                raise ValueError("No hourly data found in response.")

            weather_point = hourly[0]
            temperature = weather_point.get("temp")
            weather_desc = weather_point.get("weather", [{}])[0].get("description", "N/A")

            precipitation = 0
            precip_type = "none"
            if "rain" in weather_point:
                precipitation = weather_point["rain"].get("1h", 0)
                precip_type = "rain"
            elif "snow" in weather_point:
                precipitation = weather_point["snow"].get("1h", 0)
                precip_type = "snow"

            weather_data.append({
                "location": location,
                "date_year": timestamp1,
                "temperature": temperature,
                "weather": weather_desc,
                "precipitation_type": precip_type,
                "precipitation_mm": precipitation
            })

    except requests.RequestException as e:
            weather_data.append({
                "location": location,
                "date_year": timestamp1,
                "error": f"Request failed: {e}"
            })
    except Exception as e:
            weather_data.append({
                "location": location,
                "date_year": timestamp1,
                "error": f"Processing error: {e}"
            })

    print(weather_data)

    return weather_data


In [0]:
# call_amadeus("BLR","GOI","2025-07-29","2025-08-05",2,"INR")

In [0]:
def extract_sorted_flights_by_price_with_baggage_weight(response_json):
    results = []
    
    # Map carrier codes to full names
    carrier_dict = response_json.get("dictionaries", {}).get("carriers", {})

    for offer in response_json.get("data", []):
        itineraries = offer.get("itineraries", [])
        traveler_pricing = offer.get("travelerPricings", [])[0]
        fare_details = traveler_pricing.get("fareDetailsBySegment", [])
        price = float(offer.get("price", {}).get("total", 0.0))
        currency = offer.get("price", {}).get("currency", "INR")

        outbound_segment = itineraries[0]["segments"][0]
        inbound_segment = itineraries[1]["segments"][0]

        outbound_airline = carrier_dict.get(outbound_segment["carrierCode"], outbound_segment["carrierCode"])
        inbound_airline = carrier_dict.get(inbound_segment["carrierCode"], inbound_segment["carrierCode"])

        # Build baggage weight lookup
        baggage_lookup = {
            fare["segmentId"]: fare.get("includedCheckedBags", {}).get("weight", "N/A")
            for fare in fare_details
        }

        summary = {
            "Price": price,
            "Currency": currency,
            "Departure Time": outbound_segment["departure"]["at"],
            "Return Time": inbound_segment["departure"]["at"],
            "Departure Airline": outbound_airline,
            "Return Airline": inbound_airline,
            "Check-in Baggage Weight": {
                "Departure": f'{baggage_lookup.get(outbound_segment["id"], "N/A")}kg' if baggage_lookup.get(outbound_segment["id"]) else "N/A",
                "Return": f'{baggage_lookup.get(inbound_segment["id"], "N/A")}kg' if baggage_lookup.get(inbound_segment["id"]) else "N/A",
            }
        }

        results.append(summary)

    # Sort by price
    sorted_results = sorted(results, key=lambda x: x["Price"])
    return sorted_results



In [0]:
# def get_city_iata_code(city_name):
#     # Step 1: Get access token
#     print(f"finding iata code for {city_name}")
#     auth_response = requests.post(
#         "https://test.api.amadeus.com/v1/security/oauth2/token",
#         headers={"Content-Type": "application/x-www-form-urlencoded"},
#         data={
#             "grant_type": "client_credentials",
#             "client_id": amadeus_api_key,
#             "client_secret": amadeus_secret
#         }
#     )
#     auth_response.raise_for_status()
#     access_token = auth_response.json()["access_token"]

#     # Step 2: Search for city IATA code
#     location_response = requests.get(
#         "https://test.api.amadeus.com/v1/reference-data/locations",
#         headers={"Authorization": f"Bearer {access_token}"},
#         params={"keyword": city_name, "subType": "CITY"}
#     )
#     location_response.raise_for_status()
#     data = location_response.json().get("data", [])

#     if not data:
#         print(f"No IATA code found for {city_name}")
#         return None

#     return data[0]["iataCode"]

In [0]:
# # print(get_city_iata_code("bengaluru"))
# def lower(s):
#     result = ""
#     for char in s:
#         # Check if char is uppercase (ASCII 65–90)
#         if 'A' <= char <= 'Z':
#             # Convert to lowercase by adding 32 to ASCII value
#             result += chr(ord(char) + 32)
#         else:
#             result += char
#     return result

In [0]:
def call_amadeus(origin, destination, departure_date, return_date, no, currency):
    # or1=get_city_iata_code(lower(origin))
    # dest=get_city_iata_code(lower(destination))
    # print(f"iata codes origin - {or1}, destination - {dest}")
    or1=origin
    dest=destination
    print(f"origin is {or1}, destination is {dest}")
    auth_response = requests.post(
    "https://test.api.amadeus.com/v1/security/oauth2/token",
    data={
        "grant_type": "client_credentials",
        "client_id": amadeus_api_key,
        "client_secret": amadeus_secret
        }
    )
    access_token = auth_response.json()['access_token']

    # Search flights
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {
        "originLocationCode": or1,
        "destinationLocationCode": dest,
        "departureDate": departure_date,
        "returnDate": return_date,
        "adults": 2,
        "nonStop": "false",
        "currencyCode": currency,
        "max":4
    }
    response = requests.get(
        "https://test.api.amadeus.com/v2/shopping/flight-offers",
        headers=headers,
        params=params
    )
    
    # print(response.json())

    return extract_sorted_flights_by_price_with_baggage_weight(response.json())

In [0]:
def get_access_token():
    url = 'https://test.api.amadeus.com/v1/security/oauth2/token'
    payload = {
        'grant_type': 'client_credentials',
        'client_id': amadeus_api_key,
        'client_secret': amadeus_secret
    }

    response = requests.post(url, data=payload)
    response.raise_for_status()
    return response.json()['access_token']

In [0]:
def get_hotel_ids(city_code, radius_km):
    print("--------------------checking hotel ids--------------------")
    token = get_access_token()
    print(f"Access Token: {token}")
    url = 'https://test.api.amadeus.com/v1/reference-data/locations/hotels/by-city'
    headers = {
        'Authorization': f'Bearer {token}'
    }
    params = {
        'cityCode': city_code,
        'radius': radius_km,
        'radiusUnit': 'KM',
        # 'amenities': 'SWIMMING_POOL',  # Optional filter
        # 'ratings': '3',                # Optional filter
        'hotelSource': 'ALL'
    }

    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    data = response.json().get('data', [])
    hotel_ids = [hotel['hotelId'] for hotel in data]
    
    print(f"✅ Found {len(hotel_ids)} hotels in {city_code}")
    return hotel_ids[:20]


In [0]:
def get_hotel_offers(city_code, check_in_date, no,rooms, check_out_date):
    print("---------------inside get hotel offers--------------")
    hotel_ids=get_hotel_ids(city_code,10)
    
    return get_hotel_offers_by_ids(hotel_ids,check_in_date,no, rooms,check_out_date)


In [0]:
def get_hotel_offers_by_ids(hotel_ids, check_in_date, adults, rooms, check_out_date):
    print("--------------------checking hotel offers based on ids--------------------")
    token = get_access_token()
    print(f"Access Token: {token}")
    url = 'https://test.api.amadeus.com/v3/shopping/hotel-offers'
    headers = {
        'Authorization': f'Bearer {token}'
    }

    all_offers = []

    for hotel_id in hotel_ids:
        params = {
            'hotelIds': hotel_id,
            # 'adults': adults,
            'checkInDate': check_in_date,
            'checkOutDate': check_out_date
            # 'roomQuantity': rooms,
            # 'paymentPolicy': 'NONE',
            # 'includeClosed': 'false',
            # 'bestRateOnly': 'true',
            # 'view': 'FULL',
            # 'sort': 'PRICE'
        }

        try:
            print(f"🔍 Checking hotel ID: {hotel_id}")
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()
            offers = response.json()
            if "data" in offers and offers["data"]:
                print(f"✅ Found offers for hotel ID: {hotel_id}")
                all_offers.extend(offers["data"])
            else:
                print(f"⚠️ No offers returned for hotel ID: {hotel_id}")
        except requests.exceptions.HTTPError as e:
            print(f"❌ HTTPError for hotel ID {hotel_id}: {e}")

    if all_offers:
        return json.dumps({"data": all_offers}, indent=2)
    else:
        return json.dumps({"message": "No valid hotel offers found."}, indent=2)


In [0]:
# print(get_hotel_offers("GOI","2025-06-03",2,1,"2025-06-08"))

In [0]:
check_stays_function = {
    "name": "get_hotel_offers",
    "description": "Call this tool whenever you need to check the hotel availability and prices for the vacation destination. You need to supply the city_code, check_in_date,\
    number of heads, and number of rooms",
    "parameters": {
        "type": "object",
        "properties": {
            "city_code": {
                "type": "string",
                "description": "The IATA code for the vacation destination",
            },
            "check_in_date": {
                "type": "string",
                "description": "the date when the user will be checking into the hotel",
            },
            "no": {
                "type": "string",
                "description": "the number of heads for which reservation needs to be made, that is, how many members should the reservation be made for",
            },
            "rooms": {
                "type": "string",
                "description": "The number of rooms to be reserved as confirmed by the user",
            },
             "check_out_date": {
                "type": "string",
                "description": "the date when the user will be checking out of the hotel",
            },
        },
        "required": ["city_code","check_in_date","no","rooms","check_out_date"],
        "additionalProperties": False
    }
}

In [0]:
check_flights_function = {
    "name": "call_amadeus",
    "description": "Call this tool whenever you need to check the flight prices and other details for a return \
    trip from the origin to the destination location (where the user wants to spend his vacation). Make sure that you \
    supply the origin, destination, departure date, return date, number of tickets, and the \
    currency in which the user would like to pay. Please note the details provided will be for the return trip and NOT one-way\
                    ",
    "parameters": {
        "type": "object",
        "properties": {
            "origin": {
                "type": "string",
                "description": "Origin location for the user - his origin city or a nearby city with an airport",
            },
            "destination": {
                "type": "string",
                "description": "Destination location for the user - his vacation destination airport or airport \
                which is nearby to his vacation destination",
            },
            "departure_date": {
                "type": "string",
                "description": "the start date for the user's vacation",
            },
            "return_date": {
                "type": "string",
                "description": "the end date/ return date for the user's vacation",
            },
            "no": {
                "type": "string",
                "description": "the number of tickets to purchase",
            },
            "currency": {
                "type": "string",
                "description": "the currency in which payment is to be made",
            },
        },
        "required": ["origin","destination","departure_date","return_date","no","currency"],
        "additionalProperties": False
    }
}

In [0]:
check_weather_function = {
    "name": "check_weather",
    "description": "Call this tool whenever you need to check the weather of a location for a specific\
                    time from the previous year. The tool will require -\
                    1) the llm to supply details of one location (based on the category- beaches or mountains or any other category the user selects) \
                    and to which the user might travel to, \
                    2) the latitude and longitude of that location. \
                    3) the date_year, which basically is (vacation start date + 1 from previous year.) - this is essentially the date against which the weather conditions are to be \
                    checked. For simplicity, we would keep it as the vacation start date + 1 from the previous year. Example, if the user provides the date as june 3rd, 2025,\
                    date_year will be june 4th, 2024.\
                    This tool call will return a list of weather situations one year ago for the chosen location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "One of the locations that is near the user's location based on the\
                    category the user selects (beaches or mountains or any other destination category based on the user's choice)",
            },
            "lat": {
                "type": "string",
                "description": "The latitude of the location",
            },
            "long": {
                "type": "string",
                "description": "The longitude of the location",
            },
            "date_year": {
                "type": "string",
                "description": "The date of the previous year for which the weather needs to be fetched",
            }
        },
        "required": ["location","lat","long","date_year"],
        "additionalProperties": False
    }
}

In [0]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": check_weather_function},
        {"type": "function", "function": check_flights_function},
        {"type": "function", "function": check_stays_function}]

In [0]:
def play_audio(audio_segment):
    temp_dir = tempfile.gettempdir()
    temp_path = os.path.join(temp_dir, "temp_audio.wav")
    try:
        audio_segment.export(temp_path, format="wav")
        # time.sleep(3) # Student Dominic found that this was needed. You could also try commenting out to see if not needed on your PC
        subprocess.call([
            "ffplay",
            "-nodisp",
            "-autoexit",
            "-hide_banner",
            temp_path
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    finally:
        try:
            os.remove(temp_path)
        except Exception:
            pass
 
def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="alloy",  # Also, try replacing with onyx
        input=message
    )
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play_audio(audio)


In [0]:
# We have to write that function handle_tool_call:

def handle_tool_call1(name, args):
    location = args.get('location')
    lat = args.get('lat')
    long = args.get('long')
    date_year = args.get('date_year')
    if name.replace('"','') == "check_weather":
        weather=check_weather(location, lat, long, date_year)
            
    return weather

In [0]:
# We have to write that function handle_tool_call:

def handle_tool_call2(name, args):
    origin = args.get('origin')
    destination = args.get('destination')
    departure_date = args.get('departure_date')
    return_date = args.get('return_date')
    no = args.get('no')
    currency = args.get('currency')
    if name.replace('"','') == "call_amadeus":
        flights=call_amadeus(origin, destination, departure_date, return_date,no,currency)
    
    return flights

In [0]:
# We have to write that function handle_tool_call:

def handle_tool_call3(name, args):
    city_code = args.get('city_code')
    check_in_date = args.get('check_in_date')
    no = args.get('no')
    rooms = args.get('rooms')
    check_out_date = args.get('check_out_date')
    if name.replace('"','') == "get_hotel_offers":
        hotels=get_hotel_offers(city_code, check_in_date, no, rooms,check_out_date)
    
    return hotels

In [0]:
def chat_open_ai(history):
    messages = [{"role": "system", "content": system_message}] + history 
    response = openai.chat.completions.create(model=gpt_model, messages=messages, tools=tools)

    tool_responses = []

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        tool_calls = message.tool_calls  # renamed to avoid UnboundLocalError

        print(f"tool calls \n\n {tool_calls}")

        for tool_call in tool_calls:
            tool_id = tool_call.id
            name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)

            # Call the tool handler
            result = ""
            if name == "check_weather":
                result = handle_tool_call1(name, args)
            elif name == "call_amadeus":
                result = handle_tool_call2(name, args)
            elif name == "get_hotel_offers":
                result = handle_tool_call3(name, args)

            tool_responses.append({
                "role": "tool",
                "tool_call_id": tool_id,
                "content": json.dumps(result),
            })

        print(f"tool responses {tool_responses}")
        messages.append(message)
        messages.extend(tool_responses)  # important fix here

        response = openai.chat.completions.create(
            model=gpt_model,
            messages=messages,
            tools=tools
        )

    reply = response.choices[0].message.content
    # talker(reply)
    history += [{"role": "assistant", "content": reply}]

    return history


In [0]:
def chat(history):
    # + [{"role": "user", "content": message}]
    # if Model=="Open AI":
    history = chat_open_ai(history)
    # 

    return history

In [0]:
def listen2(history):
    import speech_recognition as sr

    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Speak now...")
        audio = r.listen(source, phrase_time_limit=30)
        text=""
        try:
            text = r.recognize_google(audio)
            print("You said:", text)
        except sr.UnknownValueError:
            print("Could not understand audio.")

        history += [{"role":"user", "content":text}]        
        return "", history

In [0]:
def get_unix_timestamp(date):
    if date is None:
        return "Please select a date."
    if isinstance(date, str):
        # Convert timestamp (string) to datetime
        date = datetime.strptime(date, "%Y-%m-%d").date()

    dt = datetime.combine(date, time(0, 0))  # Midnight UTC
    unix_timestamp = int(dt.timestamp())
    # url = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=39.099724&lon=-94.578331&dt={unix_timestamp}&appid={open_weather_api_key}"
    return unix_timestamp

In [0]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        # image_output = gr.Image(height=500)
    # with gr.Row():    
    #     date_input = gr.DateTime()
    #     output_box = gr.Textbox(label="UNIX Timestamp + API URL", lines=3)

    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        speak = gr.Button("click for voice search") 
    with gr.Row():
        clear = gr.Button("Clear")

    def listen(history):
        message, history=listen2(history)
        return message, history

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    # entry.submit(get_unix_timestamp, inputs=[date_input], outputs=[output_box])
    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        # chat, inputs=chatbot, outputs=[chatbot, image_output]
        chat, inputs=[chatbot], outputs=[chatbot]
    )
    speak.click(listen, inputs=[chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=[chatbot], outputs=[chatbot]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)